# Import Data

# Invalid Prediction

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import math

In [ ]:
# importing libraries
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import GRU, Bidirectional
from keras.optimizers import SGD
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv("Ming1Jul67_total_order.csv")
df.head(5)
df['order_completed_at'] = pd.to_datetime(df['order_completed_at'])

In [ ]:
df.info()

In [ ]:
print(df.head())
print(df.tail())

# Reshape data
### for deep learning

In [ ]:
data = df.iloc[:,1:2] 
print(data.head())
print(data.shape)

In [ ]:
# Selecting Open Price values
dataset = data.unique_order_count.values 
# Reshaping 1D to 2D array
dataset = np.reshape(dataset, (-1,1)) 
dataset.shape

# Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
# scaling dataset
dataset_scaled = scaler.fit_transform(dataset)

print(dataset_scaled[:5])

In [ ]:
X_dataset = []
y_dataset = []
for i in range(50, len(dataset_scaled)):
    X_dataset.append(dataset_scaled[i-50:i, 0])
    y_dataset.append(dataset_scaled[i, 0])
    if i <= 51:
        print(X_dataset)
        print(y_dataset)
        print()

In [ ]:
# The data is converted to Numpy array
X_dataset, y_dataset = np.array(X_dataset), np.array(y_dataset)
 
#Reshaping
X_dataset = np.reshape(X_dataset, (X_dataset.shape[0], X_dataset.shape[1],1))
y_dataset = np.reshape(y_dataset, (y_dataset.shape[0],1))
print("X_train :",X_dataset.shape,"y_train :",y_dataset.shape)

In [ ]:
from tensorflow import keras

In [ ]:
model = keras.models.load_model('../notenook/train_regressorLSTM_1Jul67.keras')

In [ ]:
# fitting the model
history = model.fit(X_dataset, y_dataset, epochs = 20, batch_size = 2)
model.summary()

In [ ]:
#list all data in history
print(history.history.keys())
#summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['accuracy'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_LSTM = model.predict(X_dataset)
y_LSTM_O = scaler.inverse_transform(y_LSTM)

In [ ]:
print(data.index)

In [ ]:
print(y_LSTM_O)

In [ ]:
#Plot for RNN predictions
plt.plot(data.index, data.unique_order_count, label = "train_data", color = "b")
plt.plot(data.index[50:],y_LSTM_O, label = "y_RNN", color = "brown")
plt.legend()
plt.title("Basic LSTM")

# Fail Forecast

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# สมมติว่าคุณมีโมเดล LSTM ที่ผ่านการเทรนแล้วชื่อ 'model'
# และข้อมูลของคุณอยู่ในตัวแปรชื่อ 'data'

# ขั้นตอนที่ 1: เตรียมข้อมูลอินพุต
scaled_data = dataset_scaled 

# กำหนดจำนวนวันที่ใช้เป็นอินพุต และจำนวนวันที่ต้องการทำนาย
n_steps = 50  # ใช้ข้อมูล 50 วันล่าสุดเป็นอินพุต
n_future = 7  # ทำนาย 7 วันถัดไป (1 สัปดาห์)

# เตรียมข้อมูล n_steps วันล่าสุดเป็นอินพุต
X_forecast = scaled_data[-n_steps:].reshape((1, n_steps, 1))

# ขั้นตอนที่ 2: ทำนายทีละวันเป็นเวลา 1 สัปดาห์
predictions = []

for _ in range(n_future):
    # ทำนายค่าสำหรับวันถัดไป
    next_day_scaled = model.predict(X_forecast)
    
    # แปลงค่าที่ทำนายกลับเป็นสเกลเดิม
    next_day = scaler.inverse_transform(next_day_scaled)[0, 0]
    
    # เพิ่มค่าที่ทำนายได้เข้าไปในลิสต์
    predictions.append(next_day)
    
    # ปรับปรุง X_forecast สำหรับการทำนายวันถัดไป
    X_forecast = np.roll(X_forecast, -1, axis=1)
    X_forecast[0, -1, 0] = next_day_scaled[0, 0]

# ตอนนี้ 'predictions' มีค่าที่ทำนายสำหรับ 7 วันถัดไป
print("ค่าที่ทำนายสำหรับ 7 วันถัดไป:")
for i, pred in enumerate(predictions, 1):
    print(f"วันที่ {i}: {pred}")

# Output Chunk Forecasting

In [ ]:
#https://stackoverflow.com/questions/69906416/forecast-future-values-with-lstm-in-python

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
pd.options.mode.chained_assignment = None
tf.random.set_seed(0)

In [ ]:
df = pd.read_csv("Ming1Jul67_total_order.csv")
df['order_completed_at'] = pd.to_datetime(df['order_completed_at'])
df.info()

In [ ]:
df.index = df["order_completed_at"]
df = df.drop(["order_completed_at"], axis=1)

In [ ]:
df.info()

In [ ]:
df.head(5)

In [ ]:
# download the data
y = df['unique_order_count'].fillna(method='ffill')
y = y.values.reshape(-1, 1)

# scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(y)
y = scaler.transform(y)

# generate the input and output sequences
n_lookback = 50  # length of input sequences (lookback period)
n_forecast = 7  # length of output sequences (forecast period)

X = []
Y = []

for i in range(n_lookback, len(y) - n_forecast + 1):
    X.append(y[i - n_lookback: i])
    Y.append(y[i: i + n_forecast])

X = np.array(X)
Y = np.array(Y)

In [ ]:
# fit the model
model = Sequential()
model.add(LSTM(50, return_sequences = True,input_shape = (n_lookback,1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(n_forecast))

model.compile(loss='mean_squared_error', optimizer='adam',metrics = ["accuracy"])
model.fit(X, Y, epochs=100, batch_size=32, verbose=1)

In [ ]:
# generate the forecasts
X_ = y[- n_lookback:]  # last available input sequence
X_ = X_.reshape(1, n_lookback, 1)

Y_ = model.predict(X_).reshape(-1, 1)
Y_ = scaler.inverse_transform(Y_)

# organize the results in a data frame
df_past = df[['unique_order_count']].reset_index()
df_past.rename(columns={'order_completed_at': 'Date', 'unique_order_count': 'Actual'}, inplace=True)
df_past['Date'] = pd.to_datetime(df_past['Date'])
df_past['Forecast'] = np.nan
df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]

df_future = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_future['Date'] = pd.date_range(start=df_past['Date'].iloc[-1] + pd.Timedelta(days=1), periods=n_forecast)
df_future['Forecast'] = Y_.flatten()
df_future['Actual'] = np.nan

results = pd.concat([df_past, df_future]).set_index('Date')

# plot the results
results.plot(title='Total Order')

In [ ]:
print(df_future)

# Valid Prediction

In [ ]:
#Initialising the model
model = Sequential()
 
#Adding LSTM layers
model.add(LSTM(50, return_sequences = True, input_shape = (50,1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
#Adding the output layer
model.add(Dense(1))
 
#Compiling the model
model.compile(optimizer = 'adam',
                      loss = 'mean_squared_error',
                      metrics = ["accuracy"])

In [ ]:
model = keras.models.load_model('train_regressorLSTM_1Jul67.keras')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
#load dataset
df = pd.read_csv("Ming2Jul67_total_order.csv")
df.head(5)
df['order_completed_at'] = pd.to_datetime(df['order_completed_at'])

# # load model 
# model = keras.models.load_model('../notenook/train_regressorLSTM_1Jul67.keras')

#preprocessing

data = df.iloc[:,1:2] 
print(data.head())
print(data.shape)

# Selecting Open Price values
dataset = data.unique_order_count.values 
# Reshaping 1D to 2D array
dataset = np.reshape(dataset, (-1,1)) 
dataset.shape

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
# scaling dataset
dataset_scaled = scaler.fit_transform(dataset)

X_dataset = []
y_dataset = []
for i in range(50, len(dataset_scaled)):
    X_dataset.append(dataset_scaled[i-50:i, 0])
    y_dataset.append(dataset_scaled[i, 0])
    if i <= 51:
        print(X_dataset)
        print(y_dataset)
        print()

# The data is converted to Numpy array
X_dataset, y_dataset = np.array(X_dataset), np.array(y_dataset)
 
#Reshaping
X_dataset = np.reshape(X_dataset, (X_dataset.shape[0], X_dataset.shape[1],1))
y_dataset = np.reshape(y_dataset, (y_dataset.shape[0],1))
print("X_train :",X_dataset.shape,"y_train :",y_dataset.shape)


# fit model with lasted data
filepath='weights_check/my_best_model.epoch{epoch:02d}-loss{loss:.2f}.keras'
checkpoint = ModelCheckpoint(filepath=filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

history = model.fit(X_dataset, y_dataset, epochs = 100, batch_size = 1, callbacks=callbacks_list)

In [ ]:
#list all data in history
print(history.history.keys())
#summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['accuracy'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'accuracy'], loc='upper left')
plt.show()

### save best model

In [ ]:
model = keras.models.load_model("weights_check\my_best_model.epoch99-loss0.00.keras")
model = model.save('train_regressorLSTM_1Jul67.keras')

In [ ]:
model = keras.models.load_model('best_check/my_best_model.epoch99-loss0.00.keras')

### Future Prediction

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# สมมติว่าคุณมีโมเดล LSTM ที่ผ่านการเทรนแล้วชื่อ 'model'

# และข้อมูลของคุณอยู่ในตัวแปรชื่อ 'data'
# Selecting Open Price values

df = pd.read_csv("Ming2Jul67_total_order.csv")
df['order_completed_at'] = pd.to_datetime(df['order_completed_at'])
data = df
dataset = data.unique_order_count.values 
# Reshaping 1D to 2D array
dataset = np.reshape(dataset, (-1,1)) 

# ขั้นตอนที่ 1: เตรียมข้อมูลอินพุต
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# กำหนดจำนวนวันที่ใช้เป็นอินพุต และจำนวนวันที่ต้องการทำนาย
n_steps = 50  # ใช้ข้อมูล 50 วันล่าสุดเป็นอินพุต
n_future = 7  # ทำนาย 7 วันถัดไป (1 สัปดาห์)

# เตรียมข้อมูล n_steps วันล่าสุดเป็นอินพุต
last_steps = scaled_data[-n_steps:]
X_forecast = last_steps.reshape((1, n_steps, 1))
print(f"init lasted 5 days {scaler.inverse_transform(X_forecast[0][-5:])}")

# ขั้นตอนที่ 2: ใช้โมเดลทำนาย
forecasted_scaled = []

for _ in range(n_future):
    # ทำนายค่าถัดไป
    next_pred = model.predict(X_forecast)
    
    # เพิ่มค่าที่ทำนายได้เข้าไปในลิสต์
    forecasted_scaled.append(next_pred[0, 0])
    
    # อัปเดต X_forecast สำหรับการทำนายครั้งถัดไป
    X_forecast = np.roll(X_forecast, -1, axis=1)
    print(f"shift {_} {scaler.inverse_transform(X_forecast[0][-5:])}")
    print(f"predict {_} = {scaler.inverse_transform(next_pred)}")
    X_forecast[0, -1, 0] = next_pred[0, 0]
    print(f"after roll {_} {scaler.inverse_transform(X_forecast[0][-5:])}")

# ขั้นตอนที่ 3: ประมวลผลการทำนาย
# แปลงค่าที่ทำนายกลับเป็นสเกลเดิม
forecasted = scaler.inverse_transform(np.array(forecasted_scaled).reshape(-1, 1))
unscaled = np.array(forecasted_scaled)
# ตอนนี้ 'forecasted' มีค่าที่ทำนายสำหรับ 7 วันถัดไป

In [ ]:
print(forecasted)

In [ ]:
print(forecasted[0])

In [ ]:
forecasted_df = pd.DataFrame(forecasted)
print(forecasted_df.head())
print(forecasted_df.info())


### forecast dataframe

In [ ]:
import pandas as pd
import numpy as np

# สมมติว่า forecasted คือ array ของผลการทำนาย
latest_date = df['order_completed_at'].max()
next_date = latest_date + pd.Timedelta(days=1)
print(next_date)

#forecast # สร้าง DataFrame
start_date = next_date
date_range = pd.date_range(start=start_date, periods=len(forecasted))
print(date_range)
forecasted_df = pd.DataFrame(forecasted)

forecasted_df = pd.DataFrame({
    'date': date_range,
    'total_order': forecasted_df[0]
})

# แสดงผล
print(forecasted_df)
print("\nข้อมูลของ DataFrame:")
print(forecasted_df.info())

In [ ]:
# print(len((X_forecast)[0]))
# print(unscaled)